In [9]:
import gym
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from PIL import Image

from collections import namedtuple, deque

import math
import random
import copy

import cv2

In [10]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "cpu"

In [11]:
class StyleNet(nn.Module):
    def __init__(self):
        super(StyleNet, self).__init__()
        vgg = torchvision.models.vgg11(pretrained=True)
        self.net = nn.Sequential()
        self.net.add_module("feature1", vgg.features[:3])
        self.net.add_module("feature2", vgg.features[3:6])
        self.net.add_module("feature3", vgg.features[6:11])
        self.net.add_module("feature4", vgg.features[11:16])
        self.net.add_module("feature5", vgg.features[16:])
        
    def forward(self, im):
        f1 = self.net.feature1(im)
        f2 = self.net.feature2(f1)
        f3 = self.net.feature3(f2)
        f4 = self.net.feature4(f3)
        f5 = self.net.feature5(f4)
        return f1, f2, f3, f4, f5

In [17]:
net = StyleNet().to(device)
net = net.eval()

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [18]:
content_img = cv2.imread('./content.jpg',)
content = cv2.resize(content_img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)[:,:,::-1] / 255
content = (content - mean) / std

#https://lonelyplanet.co.kr/magazine/articles/articleLoad/AI_00002078?page=&keyType=

content_target = net(torch.FloatTensor(content).permute(2,0,1).unsqueeze(0).to(device))
content_target = [c.detach() for c in content_target]

In [19]:
style_img = cv2.imread('./starry-night.jpg',)
style = cv2.resize(style_img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)[:,:,::-1] / 255
style = (style - mean) / std
#https://pixabay.com/illustrations/starry-night-vincent-van-gough-1093721/

style_target = net(torch.FloatTensor(style).permute(2,0,1).unsqueeze(0).to(device))
ss = []
for s in style_target:
    s = s.detach()
    a,b,c,d = s.shape
    s = s.view(a*b, c*d).contiguous()/(a*b*c*d)
    s = s.matmul(s.T)
    ss.append(s)
style_target = ss

In [21]:
img = torch.ones(content.shape).to(device)
img.uniform_()
img = torch.autograd.Variable(img, requires_grad=True)

EPOCH = 10
LR = 1
alph = 1e8
beta = 1
opt = optim.Adam([img], LR)

content_idx = 3

In [ ]:
# for epoch in range(EPOCH):
feature_map = net(torch.FloatTensor(img).permute(2,0,1).unsqueeze(0).to(device))
#     print(epoch)
#     style_loss = 0
#     for s, x in zip(style_target, feature_map):
#         a,b,c,d = x.shape
#         x_ = x.view(a*b, c*d).contiguous()/(a*b*c*d)
#         x_ = x_.matmul(x_.T)
        
#         style_loss += 1/4 * ((x_ - s) **2).mean()
#     style_loss *= alph
    
#     content_loss = beta * 1/2 * ((content_target[content_idx]- feature_map[content_idx]) ** 2).mean()
#     loss = content_loss + style_loss
#     opt.zero_grad()
#     loss.backward(retain_graph = True)
#     opt.step()    
#     print(epoch , content_loss.item(), style_loss.item())
#     img = img.clamp(0,1)